In [1]:
import numpy as np
import pandas as pd
import itertools
import random
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import warnings
import gc
import torch
import os
from copy import deepcopy
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from dataclasses import dataclass, field
import wandb
from dataclasses import asdict
from typing import Any
from scipy.stats import kendalltau
import json


GPU = "cuda:0"

In [2]:
rootdir = Path().resolve().parent.parent
inputdir = rootdir / "data" / "predict-ai-model-runtime"
embeddir = Path().resolve() / "out" / "ranknet" / "embeddings"
workdir = Path().resolve() / "out" / "finetuning"
workdir.mkdir(exist_ok=True, parents=True)

In [3]:
dataset_dict = {}

for ds in ["train", "valid", "test"]:
    records = []
    for arch, perm in itertools.product(["nlp", "xla"], ["default", "random"]):
        datadir = inputdir / f"npz_all/npz/layout/{arch}/{perm}/{ds}"
        for filepath in sorted(datadir.glob("*.npz")):
            filename = str(filepath).split("/")[-1].replace(".npz", "")
            records.append(
                {
                    "arch": arch,
                    "perm": perm,
                    "filename": filename,
                    "filepath": filepath,
                    "embed_filepath": embeddir / arch / perm / ds / f"{filename}.npz",
                }
            )
    dataset_dict[ds] = pd.DataFrame(records)

In [4]:
for ds in dataset_dict:
    indexes = []
    for i, row in dataset_dict[ds].iterrows():
        try:
            np.load(row["filepath"])
            np.load(row["embed_filepath"])
            indexes.append(i)
        except FileNotFoundError as e:
            print(row["embed_filepath"])

    dataset_dict[ds] = dataset_dict[ds].iloc[indexes].reset_index(drop=True)

In [5]:
sheet_dict = pd.read_excel(
    "/home/yamaguchi/kaggle/data/clustering-1112.xlsx", sheet_name=None
)
train_valid_dataset = pd.concat(
    [dataset_dict["train"], dataset_dict["valid"]], axis=0, ignore_index=True
)
group_dict = {}
for group_name in sheet_dict:
    if not group_name.startswith("group"):
        continue
    dfgroup = sheet_dict[group_name].query("use == 1")[["arch", "perm", "filename"]]
    dataset = dfgroup.merge(train_valid_dataset, on=["arch", "perm", "filename"])
    sheet_dict[group_name]
    print(group_name, dataset.shape, dfgroup.shape)
    if dataset.shape[0] > 0:
        group_dict[group_name] = dataset

group17 (18, 5) (18, 3)
group16 (30, 5) (30, 3)
group15 (50, 5) (50, 3)
group10 (22, 5) (22, 3)
group9 (19, 5) (19, 3)
group8 (21, 5) (21, 3)
group7 (10, 5) (10, 3)
group6 (10, 5) (10, 3)
group5 (53, 5) (53, 3)
group4 (20, 5) (20, 3)
group3 (20, 5) (20, 3)
group2 (18, 5) (18, 3)
group1 (64, 5) (64, 3)
group0 (17, 5) (17, 3)


In [6]:
def create_dataset_tensor(dataset, filename):
    dataset_as_dict = {}
    embeddings_list = []
    for i, row in dataset.iterrows():
        fileobj = np.load(row["filepath"])
        embed_fileobj = np.load(row["embed_filepath"])
        config_runtime = fileobj["config_runtime"]
        target = np.argsort(np.argsort(-config_runtime))
        embeddings = embed_fileobj["embeddings"]

        dataset_as_dict[i] = {
            "arch": row["arch"],
            "perm": row["perm"],
            "filename": row["filename"],
            "target": target,
            "X": embeddings,
        }
        embeddings_list.append(embeddings)
    embeddings_list = np.concatenate(embeddings_list, axis=0)

    emb_scl = embeddings_list.max(axis=0) - embeddings_list.min(axis=0)
    emb_mean = embeddings_list.mean(axis=0)
    del embeddings_list

    for i in dataset_as_dict:
        dataset_as_dict[i]["X"] = (dataset_as_dict[i]["X"] - emb_mean) / emb_scl

    with open(workdir / f"{filename}.json", "w") as f:
        json.dump({"xmean": emb_mean.tolist(), "xscl": emb_scl.tolist()}, f, indent=4)
    return dataset_as_dict

In [7]:
@dataclass
class Params:
    device: str
    dims: list[int] = field(default_factory=lambda: [512, 512, 512])
    epoch: int = 50
    T_max: int = 50
    eta_min: float = 1e-6
    lr: float = 1e-4
    weight_decay: float = 0
    grad_clip_max_norm: float = 1.0
    grad_clip_norm_type: float = 2.0

    sample_size: int = 1000
    batch_size: int = 8

    num_feats: int = 192


params = Params(device=GPU if torch.cuda.is_available() else "cpu")


class FineTuningDataset(Dataset):
    def __init__(
        self,
        dataset_as_dict: dict[str, Any],
        params: Params,
    ) -> None:
        self.dataset_as_dict = dataset_as_dict
        self.params = params

    @property
    def device(self) -> str:
        return self.params.device

    def __len__(self) -> int:
        return len(self.dataset_as_dict)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        dataset = self.dataset_as_dict[idx]
        num_configs = dataset["target"].shape[0]
        indexes = random.choices(list(range(num_configs)), k=self.params.sample_size)

        embeddings = torch.tensor(
            dataset["X"][indexes, :],
            dtype=torch.float32,
        ).to(self.device)
        target = torch.tensor(
            dataset["target"][indexes],
            dtype=torch.float32,
        ).to(self.device)

        return embeddings, target

    def get_info(self, idx):
        dataset = self.dataset_as_dict[idx]
        return dataset["arch"], dataset["perm"], dataset["filename"]


class MLP(torch.nn.Module):
    def __init__(
        self,
        params: Params,
    ) -> None:
        super().__init__()
        self.params = params

        dims = [params.num_feats] + self.params.dims
        fc_layer = []
        for i in range(len(dims) - 1):
            fc_layer += [
                nn.Linear(
                    in_features=dims[i],
                    out_features=dims[i + 1],
                ),
                nn.ReLU(),
            ]
        fc_layer += [
            nn.Linear(
                in_features=dims[-1],
                out_features=1,
            ),
        ]

        self.net = nn.Sequential(*fc_layer)
        self.to(self.params.device)

    def forward(self, x) -> torch.Tensor:
        return self.net(x).squeeze()

In [8]:
def rankNet(y_pred, y_true):
    """
    RankNet loss introduced in "Learning to Rank using Gradient Descent".
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :return: loss value, a torch.Tensor
    """
    y_pred = y_pred.clone()
    y_true = y_true.clone()

    document_pairs_candidates = list(itertools.combinations(range(y_true.shape[1]), 2))

    pairs_true = y_true[:, document_pairs_candidates]
    selected_pred = y_pred[:, document_pairs_candidates]

    true_diffs = pairs_true[:, :, 0] - pairs_true[:, :, 1]
    pred_diffs = selected_pred[:, :, 0] - selected_pred[:, :, 1]

    the_mask = (true_diffs > 0) & (~torch.isinf(true_diffs))
    pred_diffs = pred_diffs[the_mask]

    true_diffs = (true_diffs > 0).type(torch.float32)
    true_diffs = true_diffs[the_mask]

    return nn.BCEWithLogitsLoss()(pred_diffs, true_diffs)

In [9]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def to_cpu_numpy(
    params: Params, pred: torch.Tensor, truth: torch.Tensor
) -> tuple[np.ndarray, np.ndarray]:
    if params.device == GPU:
        pred_ = pred.cpu().detach().numpy()
        truth_ = truth.cpu().detach().numpy()
        torch.cuda.empty_cache()
    else:
        pred_ = pred.detach().numpy()
        truth_ = truth.detach().numpy()
    return pred_, truth_


def train(params, dataset_as_dict, savedir):
    train_dataset = FineTuningDataset(dataset_as_dict=dataset_as_dict, params=params)
    valid_dataset = train_dataset
    train_dataloader = DataLoader(
        train_dataset, batch_size=params.batch_size, shuffle=True
    )

    model = MLP(params=params)
    optimizer = torch.optim.Adam(
        model.parameters(), lr=params.lr, weight_decay=params.weight_decay
    )
    scheduler = CosineAnnealingLR(
        optimizer=optimizer, T_max=params.T_max, eta_min=params.eta_min
    )

    num_train_log, num_valid_log = 0, 0
    for epoch in range(params.epoch):
        model.train()

        num_iters = len(train_dataloader)
        pbar = tqdm(range(num_iters))

        for i_iter, (X, target) in enumerate(train_dataloader):
            pred = model(X)

            if (len(pred.shape) == 1) or (len(target.shape) == 1):
                pred, target = pred.reshape(1, -1), target.reshape(1, -1)
            loss = rankNet(pred, target)
            loss.backward()
            nn.utils.clip_grad_norm_(
                model.parameters(),
                max_norm=params.grad_clip_max_norm,
                norm_type=params.grad_clip_norm_type,
            )
            optimizer.step()
            scheduler.step(epoch + i_iter / num_iters)
            optimizer.zero_grad()

            pred, target = to_cpu_numpy(params, pred, target)

            scores = []
            for pred_, target_ in zip(pred, target):
                score = kendalltau(target_, pred_).correlation
                wandb.log(
                    {
                        "epoch": epoch,
                        "iter": i_iter,
                        "lr": scheduler.get_last_lr()[0],
                        "train/score": score,
                        "train/loss": loss,
                        "train/pred": pred,
                        "train/target": target,
                        "train/count": num_train_log,
                    }
                )
                num_train_log += 1
                scores.append(score)
            scores = np.array(scores)
            pbar.set_description(
                f"[{epoch + 1}] score={np.mean(scores[~np.isnan(scores)]):.3f} loss={loss.item():5f}"
            )
            pbar.update(1)

        model.eval()
        losses, scores = [], []
        for i_graph in range(len(valid_dataset)):
            arch, perm, filename = valid_dataset.get_info(i_graph)
            X, target = valid_dataset[i_graph]
            pred = model(X)
            loss = rankNet(pred.reshape(1, -1), target.reshape(1, -1))
            graph_loss = loss.item()
            pred, target = to_cpu_numpy(params, pred, target)
            score = kendalltau(target, pred).correlation
            if np.isnan(score):
                import pdb

                pdb.set_trace()

            wandb.log(
                {
                    "epoch": epoch,
                    "iter": i_iter,
                    "lr": scheduler.get_last_lr()[0],
                    "valid/score": score,
                    "valid/loss": graph_loss,
                    "valid/pred": pred,
                    "valid/target": target,
                    "valid/count": num_valid_log,
                }
            )
            num_valid_log += 1
            losses.append(graph_loss)
            scores.append(score)
        losses = np.array(losses)
        scores = np.array(scores)
        print(
            f"[{epoch + 1}] valid score={np.mean(scores[~np.isnan(scores)]):.3f} valid loss={np.mean(losses[~np.isnan(losses)]):5f}"
        )

        torch.save(model.state_dict(), savedir / f"epoch{epoch + 1}_model.pt")

In [10]:
seed = 43

for group_name in ["group10", "group9", "group3", "group2"]:
    dataset = group_dict[group_name]
    dataset_as_dict = create_dataset_tensor(dataset=dataset, filename=group_name)

    exptname = f"1108-勾配クリップ-ミスマッチ-ranknet-finetuning-{group_name}-seed{seed}"

    wandb.init(
        project="predict-ai-model-runtime-for-sun-scan-clan",
        config={
            "params": asdict(params),
        },
        name=exptname,
    )

    seed_everything(seed)

    savedir = workdir / f"{group_name}" / f"seed{seed}"
    savedir.mkdir(exist_ok=True, parents=True)
    train(params=params, dataset_as_dict=dataset_as_dict, savedir=savedir)

    wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zuuuubo-tetsu (sun-scan-clan). Use `wandb login --relogin` to force relogin


  0%|          | 0/3 [00:00<?, ?it/s]

/home/yamaguchi/.pyenv/versions/3.11.5/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[1] valid score=0.546 valid loss=0.687018


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

[49] valid score=0.588 valid loss=0.417690


  0%|          | 0/3 [00:00<?, ?it/s]

[50] valid score=0.580 valid loss=0.421207


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
iter,▅▅██▅███▁██▁███▁▅█▁▅▅██▅██▅▁██▁███▁▅█▁▅█
lr,███████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/count,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/loss,████▇▇▇▇▆▆▅▅▅▄▄▃▄▄▁▃▂▃▇▂▂▂▂▁▃▁▂▂▂▂▂▁▂▂▃▃
train/score,▁▆▅▄▇▇▄▇▇▇▅▄▆▅▆▅▂▂▇▂█▇▂▇▄█▇▅▄▇█▇█▄▇██▄▅▄
valid/count,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
valid/loss,█████▇▇▆▇▅▅▇▄▄▄▆▆▃▆▆▆▂▂▆▂▂▆▅▂▁▅▃▁▂▅▅▂▅▅▂
valid/score,▁▂▆▆▂▆▆▇▄▆▇▅▆▇▆▄▂▆▃▂▂▇▇▂▇▇▂▅▇█▅▆█▇▄▂▇▄▃▇
epoch,49
iter,2


In [63]:
# for seed in [43, 44, 45, 46, 47]:
#     for group_name in group_dict:
#         dataset = group_dict[group_name]
#         dataset_as_dict = create_dataset_tensor(dataset=dataset, filename=group_name)

#         exptname = f"1108-勾配クリップ-ミスマッチ-ranknet-finetuning-{group_name}-seed{seed}"

#         wandb.init(
#             project="predict-ai-model-runtime-for-sun-scan-clan",
#             config={
#                 "params": asdict(params),
#             },
#             name=exptname,
#         )

#         seed_everything(seed)

#         savedir = workdir / f"{group_name}" / f"seed{seed}"
#         savedir.mkdir(exist_ok=True, parents=True)
#         train(params=params, dataset_as_dict=dataset_as_dict, savedir=savedir)

#         wandb.finish()

epoch,▁▁▁▁▁▁
iter,▁▁▁▁▁▁
lr,▁▁▁▁▁▁
train/count,▁▂▄▅▇█
train/loss,▁▁▁▁▁▁
train/score,▆▁█▇▃▇
epoch,0
iter,0
lr,0.0001
train/count,5
train/loss,0.69315


  0%|          | 0/3 [00:00<?, ?it/s]

[1] valid score=0.190 valid loss=0.692820


  0%|          | 0/3 [00:00<?, ?it/s]

[2] valid score=0.203 valid loss=0.692683


...

  0%|          | 0/8 [00:00<?, ?it/s]

[50] valid score=0.651 valid loss=0.309971


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
iter,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
train/count,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/loss,██▇▇▆▄▄▃▃▃▂▁▂▄▁▄▃▂▃▂▃▂▂▃▂▃▃▂▁▂▂▂▂▄▃▃▂▂▃▂
train/score,▅▆▇▁▂▆▂▁▇▂▂█▃▂▃▇▂▄▃█▂███▄▃▃▃██▃██▆█▆█▆▅▃
valid/count,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
valid/loss,███▇▄▃▄▄▂▁▃▃▁▃▄▁▁▄▄▁▁▄▄▁▁▄▁▁▄▃▁▁▃▄▁▁▄▄▁▄
valid/score,▃▄▁▁▅▅▂▂▇█▃▃█▃▃██▃▃██▃▃██▃██▃▃██▃▃██▃▃█▃
epoch,49
iter,7


  0%|          | 0/3 [00:00<?, ?it/s]

IndexError: too many indices for tensor of dimension 1